In [1]:
#!pip install tensorflow==2.5.0
#!pip install tensorflow_datasets
#!pip install ipywidgets

In [1]:
import tensorflow as tf
print(tf.__version__)

2021-12-12 18:13:02.023083: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-12 18:13:02.023134: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.5.0


In [2]:
import tensorflow_datasets as tfds

In [3]:
imdb, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)

2021-12-12 18:13:32.224578: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-12 18:13:32.224619: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-12 18:13:32.224648: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (amanpreet-singh): /proc/driver/nvidia/version does not exist


In [4]:
train_data = imdb['train']
test_data = imdb['test']

In [7]:
tokenizer = info.features['text'].encoder

In [9]:
print(type(imdb))
print(imdb.keys())
print(type(imdb['train']))
print(type(info))
print(type(info.features))
print(info.features.keys())
print(info.features['text'])
print(info.features['text'].encoder)
print(tokenizer.vocab_size)

<class 'dict'>
dict_keys(['train', 'test', 'unsupervised'])
<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>
<class 'tensorflow_datasets.core.dataset_info.DatasetInfo'>
<class 'tensorflow_datasets.core.features.features_dict.FeaturesDict'>
dict_keys(['text', 'label'])
Text(shape=(None,), dtype=tf.int64, encoder=<SubwordTextEncoder vocab_size=8185>)
<SubwordTextEncoder vocab_size=8185>
8185


In [10]:
test_string = "That movie i saw the other day was fantabulous"
print("test string: ", test_string)
tokenized_string = tokenizer.encode(test_string)
print("tokenized string: ", tokenized_string)
decoded_string = tokenizer.decode(tokenized_string)
print("decoded string: ", decoded_string)

test string:  That movie i saw the other day was fantabulous
tokenized string:  [444, 27, 131, 284, 1, 108, 414, 18, 1042, 4779, 5872]
decoded string:  That movie i saw the other day was fantabulous


In [11]:
for token in tokenized_string:
  print('{} ----> {}'.format(token, tokenizer.decode([token])))

444 ----> That 
27 ----> movie 
131 ----> i 
284 ----> saw 
1 ----> the 
108 ----> other 
414 ----> day 
18 ----> was 
1042 ----> fan
4779 ----> tab
5872 ----> ulous


In [12]:
# saving the tokenizer
import pickle
with open('tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_data.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(train_dataset))
test_dataset = test_data.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(test_data))

In [14]:
embedding_dim = 64
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, embedding_dim),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
global_average_pooling1d (Gl (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 390       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 524,237
Trainable params: 524,237
Non-trainable params: 0
_________________________________________________________________


In [16]:
num_epochs = 10

In [17]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [18]:
history = model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)

Epoch 1/10


2021-12-12 13:52:48.697181: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-12 13:52:48.733025: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2394800000 Hz


391/391 [==============================] - 25s 56ms/step - loss: 0.6635 - accuracy: 0.6318 - val_loss: 0.5981 - val_accuracy: 0.7801
Epoch 2/10
391/391 [==============================] - 20s 51ms/step - loss: 0.4852 - accuracy: 0.8232 - val_loss: 0.4168 - val_accuracy: 0.8527
Epoch 3/10
391/391 [==============================] - 20s 50ms/step - loss: 0.3450 - accuracy: 0.8769 - val_loss: 0.3453 - val_accuracy: 0.8708
Epoch 4/10
391/391 [==============================] - 19s 49ms/step - loss: 0.2818 - accuracy: 0.8999 - val_loss: 0.3188 - val_accuracy: 0.8768
Epoch 5/10
391/391 [==============================] - 20s 50ms/step - loss: 0.2478 - accuracy: 0.9119 - val_loss: 0.3102 - val_accuracy: 0.8778
Epoch 6/10
391/391 [==============================] - 19s 49ms/step - loss: 0.2234 - accuracy: 0.9198 - val_loss: 0.3110 - val_accuracy: 0.8770
Epoch 7/10
391/391 [==============================] - 17s 44ms/step - loss: 0.2046 - accuracy: 0.9268 - val_loss: 0.3208 - val_accuracy: 0.8706
Epo

In [37]:
#test an examples on the trained model
test_review = "That movie i saw the other day was fantastic"
tokenized_string = tokenizer.encode(test_review)
import numpy as np
prediction = model.predict(tokenized_string)
prediction[np.argmax(prediction)][0]

1.0

In [38]:
## save the model into well defined directory
import time
ts = int(time.time())
file_path = f"sentiment_classifier/{ts}/"
model.save(filepath=file_path, save_format='tf')

2021-12-12 14:07:43.563816: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: sentiments_classifier/1639298263/assets


INFO:tensorflow:Assets written to: sentiments_classifier/1639298263/assets
